In [2]:
from ipyleaflet import Map, Marker, basemaps, GeoJSON, FullScreenControl, DivIcon, Polyline, CircleMarker
import pandas as pd
import json
from PIL import Image
import networkx as nx

In [4]:
stations = pd.read_csv("stations_of_interest.csv")

In [5]:
center = (52.198, 5.537)
m = Map(center=center, zoom=9, basemap=basemaps.CartoDB.Positron)

with open('provincie_2025.geojson', 'r') as f:
    data = json.load(f)

geo_json = GeoJSON(data=data, style={'fillColor': 'black', 'weight': 0, 'fillOpacity': 1})
m.add(geo_json)

m.add(FullScreenControl())

for _, station in stations.iterrows():
    icon = DivIcon(html='<div class="markerr"></div>', center=(station['lat'], station['lng']), draggable=False, bg_pos=(0,0), icon_size=(1,1))
    marker = Marker(location=(station['lat'], station['lng']), icon=icon)
    m.add(marker)

display(m)

m.save('map.html', title='Map')

Map(center=[52.198, 5.537], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [6]:
outline = Image.open("outline.png")
outline.show()

In [7]:
output = []
n_stations = 0

In [66]:
for y in range(outline.height):
    row = []
    for x in range(outline.width):
        r, g, b, a = outline.getpixel((x, y))
        value = (1 if g > 0 else 0) + (2 if r > 0 else 0)
        n_stations += r > 0
        row.append(value)
    output.append(row)

In [70]:
print(n_stations, len(stations))

34 34


In [73]:
with open('output.json', 'w+') as o:
    json.dump(output, o)

In [58]:
test = Image.new('RGBA', (outline.width, outline.height))

for y in range(test.height):
    for x in range(test.width):
        value = output[y][x]
        r = 255 if value == 2 else 0
        g = 255 if value == 1 else 0
        b = 0
        a = 255 if value > 0 else 0
        test.putpixel((x, y), (r, g, b, a))

In [59]:
test.show()

In [60]:
outline.width, outline.height

(826, 965)

In [23]:
nodes = pd.read_csv("nodes.csv")
edges = pd.read_csv("edges.csv")

In [43]:
m2 = Map(center=center, zoom=9, basemap=basemaps.CartoDB.Positron)
G = nx.Graph()

for _, row in nodes.iterrows():
    G.add_node(row['name'], station=row['station'], lat=row['lat'], lng=row['lng'])

    if (stations['code'].eq(row['name'])).any():
        marker = Marker(location=(row['lat'], row['lng']), draggable=False)
    else:
        marker = CircleMarker(location=(row['lat'], row['lng']), draggable=False, radius=2, color="black", fill_color="black")
    m2.add(marker)

lat = nx.get_node_attributes(G, 'lat')
lng = nx.get_node_attributes(G, 'lng')
for _, row in edges.iterrows():
    G.add_edge(row['from'], row['to'])
    locs = [[lat[row['from']], lng[row['from']]], [lat[row['to']], lng[row['to']]]]
    line = Polyline(locations=locs, color='blue', weight=2)
    m2.add(line)

print(G)
m2.add(FullScreenControl())

display(m2)

Graph with 395 nodes and 434 edges


Map(center=[52.198, 5.537], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…